<a href="https://colab.research.google.com/github/robonesky/oreilly/blob/main/RM%20Lab%20Notes%20Advanced%20ML-%20part%20II.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install pandas_ta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=da3516f70dc5a054e57b482996437fe6630e6a71e7e7f24f812061aac167a524
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


In [4]:
%pip install boruta

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 2.2 MB/s eta 0:00:00


In [5]:
%pip install keras_tuner

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.1/176.1 kB 4.3 MB/s eta 0:00:00


In [6]:
# Machine Learning Workshop - I
# Kannan Singaravelu

# basic imports
import os, random
import pandas as pd
import numpy as np
import datetime as dt
import pandas_ta as ta
from pathlib import Path

# import boruta
from boruta import BorutaPy

# warnings
import warnings
warnings.filterwarnings('ignore')

# plotting & outputs
from pprint import pprint
import matplotlib.pyplot as plt
plt.style.use('seaborn')

# functions from helper
# from src.helper import *

# import custom transformer
# from helper import DayTransformer, TimeTransformer

# sklearn imports
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit, cross_val_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

# metrics
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import auc, roc_curve #, plot_roc_curve # plot_confusion_matrix

# import classifiers
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier

# tensorflow
import tensorflow as tf
from tensorflow.keras.utils import plot_model
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.metrics import BinaryAccuracy, Accuracy, AUC, Precision, Recall
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
from tensorflow.keras.layers import Dropout, Dense, Flatten
from tensorflow.keras.layers import LSTM, BatchNormalization

# kerastuner
import keras_tuner as kt
from kerastuner import HyperParameter, HyperParameters
from kerastuner.tuners import RandomSearch, BayesianOptimization, Hyperband

from sklearn.base import BaseEstimator, TransformerMixin

# Machine Learning Workshop - I by Kannan Singaravelu
# November 2021


In [7]:
# Machine Learning Workshop - I
# Kannan Singaravelu

import pandas as pd
import numpy as np
import random
import tensorflow as tf
from sklearn.base import BaseEstimator, TransformerMixin


# define seed
def set_seeds(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)


# create custom day transformer
class DayTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.data = pd.DataFrame(
            {
        'WeekDay': ["Monday","Tuesday","Wednesday","Thursday","Friday","Saturday","Sunday"]
            }
        )
        self.daysnum = np.array(self.data.index+1)

        print(f'self.daysnum is: {self.daysnum}')

        return self


    def transform(self, X): # X is a dataframe
        Xt = X.copy()
        pi = np.pi
        num = Xt.index.weekday+1

        Xt['dsin'] = np.sin(2 * pi * num / np.max(self.daysnum))
        Xt['dcos'] = np.cos(2 * pi * num / np.max(self.daysnum))
        Xt = Xt.drop(['days'], axis=1)

        return Xt


# create custom time transformer
class TimeTransformer(BaseEstimator, TransformerMixin):

    def __init__(self):
        pass

    def fit(self, X, y=None):
        self.data = pd.DataFrame(
            {
        'DayParts': ["afternoon","morning","noon"]
            }
        )
        self.timenum = np.array(self.data.index+1)
        return self


    def transform(self, X):
        Xt = X.copy()
        pi = np.pi
        num = Xt.hours.apply(lambda x: 1 if x=='afternoon' else (2 if x=='morning' else 3))
        Xt['tsin'] = np.sin(2 * pi * num / np.max(self.timenum))
        Xt['tcos'] = np.cos(2 * pi * num / np.max(self.timenum))
        Xt = Xt.drop(['hours'], axis=1)

        return Xt


# create function to read locally stored file
def getdata(filename):
    df = pd.read_csv('./'+filename+'.csv')
    df.datetime = pd.to_datetime(df.datetime)
    df = (
        df.set_index('datetime', drop=True)
        .drop('symbol', axis=1)
    )

    # add days
    df['days'] = df.index.day_name()

    # add dayparts
    df['hours'] = df.index.hour
    df['hours'] = df['hours'].apply(daypart)

    return df


# create function to group trade hours
def daypart(hour):
    if hour in [9,10,11]:
        return "morning"
    elif hour in [12,13]:
        return "noon"
    elif hour in [14,15,16,17,18,19]:
        return "afternoon"


# class weight function
def cwts(dfs):
    c0, c1 = np.bincount(dfs)
    w0=(1/c0)*(len(dfs))/2
    w1=(1/c1)*(len(dfs))/2
    return {0: w0, 1: w1}

# Machine Learning Workshop - I by Kannan Singaravelu
# November 2021

In [9]:
df = getdata('NFUT1H')
df.drop(['days','hours'], axis=1, inplace = True)
df

,open,high,low,close,volume
datetime,,,,,
2018-08-16 14:00:00,11410.00,11428.45,11377.00,11402.00,1475100.0
2018-08-16 15:00:00,11401.50,11420.45,11395.35,11417.00,960075.0
2018-08-17 09:00:00,11439.75,11475.00,11420.00,11470.20,1789950.0
2018-08-17 10:00:00,11469.55,11491.65,11462.30,11488.20,1108500.0
2018-08-17 11:00:00,11488.00,11493.00,11483.50,11487.95,470550.0
...,...,...,...,...,...
2021-07-14 11:00:00,15839.40,15854.95,15827.70,15848.40,483650.0
2021-07-14 12:00:00,15849.20,15878.00,15839.00,15874.20,667000.0
2021-07-14 13:00:00,15874.30,15893.25,15847.45,15874.00,665450.0


In [10]:
# standard stuff so far:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
open,5000.0,1.193472e+04,1738.573012,7611.55,10895.30,11580.925,1.227634e+04,15921.15
high,5000.0,1.196289e+04,1735.832211,7630.05,10920.00,11598.200,1.229174e+04,15930.60
low,5000.0,1.190535e+04,1739.916371,7526.00,10873.90,11561.825,1.226578e+04,15897.05
close,5000.0,1.193518e+04,1738.400558,7580.10,10895.45,11580.450,1.227681e+04,15918.90
volume,5000.0,1.707417e+06,984467.503150,272800.00,1045856.25,1508662.500,2.107200e+06,13178175.00


In [11]:
df.isnull().sum()

open      0
high      0
low       0
close     0
volume    0
dtype: int64

In [15]:
%pip install TA-lib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 272.6/272.6 kB 5.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for TA-lib (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for TA-lib
Failed to build TA-lib
ERROR: Could not build wheels for TA-lib, which is required to install pyproject.toml-based projects


In [16]:
# add all the pandas_ta factors:

df.ta.strategy('all')

0it [00:00, ?it/s]

[X] Please install TA-Lib to use 2crows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3blackcrows. (pip install TA-Lib)
[X] Please install TA-Lib to use 3inside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3linestrike. (pip install TA-Lib)
[X] Please install TA-Lib to use 3outside. (pip install TA-Lib)
[X] Please install TA-Lib to use 3starsinsouth. (pip install TA-Lib)
[X] Please install TA-Lib to use 3whitesoldiers. (pip install TA-Lib)
[X] Please install TA-Lib to use abandonedbaby. (pip install TA-Lib)
[X] Please install TA-Lib to use advanceblock. (pip install TA-Lib)
[X] Please install TA-Lib to use belthold. (pip install TA-Lib)
[X] Please install TA-Lib to use breakaway. (pip install TA-Lib)
[X] Please install TA-Lib to use closingmarubozu. (pip install TA-Lib)
[X] Please install TA-Lib to use concealbabyswall. (pip install TA-Lib)
[X] Please install TA-Lib to use counterattack. (pip install TA-Lib)
[X] Please install TA-Lib to use darkcloudcover. (pip instal

131it [00:11, 11.82it/s]


In [13]:
df

,open,high,low,close,volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,VIDYA_14,VTXP_14,VTXM_14,VWAP_D,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30
datetime,,,,,,,,,,,,,,,,,,,,,
2018-08-16 14:00:00,11410.00,11428.45,11377.00,11402.00,1475100.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11402.483333,NaN,11402.3625,NaN,NaN,NaN,NaN
2018-08-16 15:00:00,11401.50,11420.45,11395.35,11417.00,960075.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11405.814771,NaN,11412.4500,NaN,NaN,NaN,NaN
2018-08-17 09:00:00,11439.75,11475.00,11420.00,11470.20,1789950.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11455.066667,NaN,11458.8500,NaN,NaN,NaN,NaN
2018-08-17 10:00:00,11469.55,11491.65,11462.30,11488.20,1108500.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11464.876401,NaN,11482.5875,NaN,NaN,NaN,NaN
2018-08-17 11:00:00,11488.00,11493.00,11483.50,11487.95,470550.0,11447.470000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,11468.127037,NaN,11488.1000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-07-14 11:00:00,15839.40,15854.95,15827.70,15848.40,483650.0,15822.373333,15868.634054,15776.112613,46.260721,15673.649601,...,15788.343228,0.965165,0.948987,15814.390972,15811.032804,15844.8625,-3.317296,15822.955455,15833.253804,1.585121
2021-07-14 12:00:00,15849.20,15878.00,15839.00,15874.20,667000.0,15831.223333,15877.000006,15785.446661,45.776673,15679.779601,...,15790.887752,0.997277,0.906758,15824.989930,15821.661868,15866.3500,-1.723356,15833.830909,15848.016749,1.969462
2021-07-14 13:00:00,15874.30,15893.25,15847.45,15874.00,665450.0,15839.070000,15884.848228,15793.291772,45.778228,15689.567309,...,15795.938018,1.197067,0.843773,15833.209972,15830.274880,15872.1750,-8.324324,15843.037273,15869.104613,1.825032


In [25]:
data = df.copy()

data['predict'] = np.where(
    data['close'].pct_change(-5) > 0, 1, 0
)

# drop unwanted columns:
data.drop(['HILOl_13_21', 'HILOs_13_21', 'PSARl_0.02_0.2', 'PSARs_0.02_0.2', 'PSARaf_0.02_0.2', 'QQEl_14_5_4.236', 'QQEs_14_5_4.236', 'SUPERTl_7_3.0', 'SUPERTs_7_3.0'], axis=1, inplace=True)
data = data[200:] #why are we only looking at the first 200 rows?

# we actually have some nulls - I'm not sure Kannan has noticed them though!
data.isna().sum().loc[lambda x: x != 0]

DPO_20    11
ICS_26    26
dtype: int64

In [26]:
data = data.bfill(axis=1) # backfill
data = data[:-5] # handle 5-day ahead prediction
data.tail()

,open,high,low,close,volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,VTXP_14,VTXM_14,VWAP_D,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30,predict
datetime,,,,,,,,,,,,,,,,,,,,,
2021-07-13 13:00:00,15818.15,15837.20,15812.20,15830.00,673200.0,15800.293333,15848.678457,15751.908210,48.385124,15634.304051,...,1.035491,0.895754,15798.646941,15761.969527,15827.3500,-4.006678,15784.206364,15819.625860,0.982370,0.0
2021-07-13 14:00:00,15828.05,15833.00,15805.20,15820.25,644600.0,15806.766667,15853.779449,15759.753885,47.012782,15638.623930,...,1.045413,0.928001,15801.278794,15764.851027,15819.6750,-9.432387,15793.010909,15828.866612,0.836754,0.0
2021-07-13 15:00:00,15820.60,15842.55,15820.50,15833.95,914900.0,15818.680000,15864.045263,15773.314737,45.365263,15651.286840,...,1.012943,0.900402,15805.999774,15770.633931,15832.7375,-4.647393,15803.811818,15833.181774,1.098477,0.0
2021-07-14 09:00:00,15805.10,15822.15,15777.05,15784.00,1051050.0,15816.890000,15863.024246,15770.755754,46.134246,15659.865415,...,0.921375,0.981320,15794.400000,15785.025623,15791.8000,-31.640097,15804.604545,15818.039633,0.296406,0.0
2021-07-14 10:00:00,15783.10,15847.45,15779.05,15839.40,903450.0,15818.930000,15866.548629,15771.311371,47.618629,15662.708872,...,0.908731,0.966891,15807.142443,15800.835682,15826.3250,-4.237957,15814.251818,15823.632427,1.407996,0.0


In [27]:
data.isna().sum().loc[lambda x: x != 0]

Series([], dtype: int64)

In [29]:
# address class imbalance
c = data['predict'].value_counts()
c

0.0    2621
1.0    2174
Name: predict, dtype: int64

In [30]:
def cwts(dfs):
  c0, c1 = np.bincount(dfs['predict'])
  w0 = (1/c0) * (len(df))/2
  w1 = (1/c1) * (len(df))/2

  ret = {0:w0, 1:w1}

  print (ret)

  return ret

In [31]:
class_weight = cwts(data)
class_weight

{0: 0.9538344143456696, 1: 1.1499540018399264}


{0: 0.9538344143456696, 1: 1.1499540018399264}

In [32]:
# With the calculated weights, both classes gain equal weight
class_weight[0] * c[0], class_weight[1] * c[1]

(2500.0, 2500.0)

In [33]:
# feature specification
X = data.drop('predict', axis=1)
feature_names = X.columns
X.tail()

,open,high,low,close,volume,ABER_ZG_5_15,ABER_SG_5_15,ABER_XG_5_15,ABER_ATR_5_15,ACCBL_20,...,VIDYA_14,VTXP_14,VTXM_14,VWAP_D,VWMA_10,WCP,WILLR_14,WMA_10,ZL_EMA_10,ZS_30
datetime,,,,,,,,,,,,,,,,,,,,,
2021-07-13 13:00:00,15818.15,15837.20,15812.20,15830.00,673200.0,15800.293333,15848.678457,15751.908210,48.385124,15634.304051,...,15783.321445,1.035491,0.895754,15798.646941,15761.969527,15827.3500,-4.006678,15784.206364,15819.625860,0.982370
2021-07-13 14:00:00,15828.05,15833.00,15805.20,15820.25,644600.0,15806.766667,15853.779449,15759.753885,47.012782,15638.623930,...,15784.553405,1.045413,0.928001,15801.278794,15764.851027,15819.6750,-9.432387,15793.010909,15828.866612,0.836754
2021-07-13 15:00:00,15820.60,15842.55,15820.50,15833.95,914900.0,15818.680000,15864.045263,15773.314737,45.365263,15651.286840,...,15786.552365,1.012943,0.900402,15805.999774,15770.633931,15832.7375,-4.647393,15803.811818,15833.181774,1.098477
2021-07-14 09:00:00,15805.10,15822.15,15777.05,15784.00,1051050.0,15816.890000,15863.024246,15770.755754,46.134246,15659.865415,...,15786.549918,0.921375,0.981320,15794.400000,15785.025623,15791.8000,-31.640097,15804.604545,15818.039633,0.296406
2021-07-14 10:00:00,15783.10,15847.45,15779.05,15839.40,903450.0,15818.930000,15866.548629,15771.311371,47.618629,15662.708872,...,15787.198462,0.908731,0.966891,15807.142443,15800.835682,15826.3250,-4.237957,15814.251818,15823.632427,1.407996
